### R-CNN Implementation Using Keras

## Step 1: read in dataset and draw bounding boxes on images

In [5]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import pandas as pd
from PIL import Image
from PIL.ImageDraw import Draw


## Step 2: start building model

In [6]:
width = 416
height = 416
num_classes = 3
classes = ["Crab", "Shrimp", "Lobster"]

In [11]:
def load_and_process_data(csv_file, image_dir):
    image_records = pd.read_csv(csv_file)
    image_path = os.path.join(os.getcwd(), image_dir)
    # print(image_path)

    images = []
    targets = []
    labels = []

    for index, row in image_records.iterrows():
        (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row

        image_fullpath = os.path.join(image_path, filename)
        img = keras.preprocessing.image.load_img(image_fullpath)
        img_arr = keras.preprocessing.image.img_to_array(img)
        # print(img_arr.shape)

        xmin = round(xmin/ width, 2)
        ymin = round(ymin/ height, 2)
        xmax = round(xmax/ width, 2)
        ymax = round(ymax/ height, 2)

        images.append(img_arr)
        targets.append((xmin, ymin, xmax, ymax))
        labels.append(classes.index(class_name))

    return images, targets, labels

In [12]:
# load and preprocess datasets
TRAINING_CSV_FILE = '../shellfish_dataset/train/_annotations.csv'
VALIDATION_CSV_FILE = '../shellfish_dataset/valid/_annotations.csv'
TESTING_CSV_FILE = '../shellfish_dataset/test/_annotations.csv'

TRAINING_IMAGE_DIR = '../shellfish_dataset/train'
VALIDATION_IMAGE_DIR = '../shellfish_dataset/valid'
TESTING_IMAGE_DIR = '../shellfish_dataset/test'

train_images, train_targets, train_labels = load_and_process_data(TRAINING_CSV_FILE, TRAINING_IMAGE_DIR)
validation_images, validation_targets, validation_labels = load_and_process_data(VALIDATION_CSV_FILE, VALIDATION_IMAGE_DIR)
test_images, test_targets, test_labels = load_and_process_data(TESTING_CSV_FILE, TESTING_IMAGE_DIR)

C:\Users\black\OneDrive - The George Washington University\Fall 2023\CSCI 4366\Final Project\nn_f23_shellfish\../shellfish_dataset/train\d66fe3c0f7d5a396_jpg.rf.0d947e33ef400474603dc78f438e2101.jpg
C:\Users\black\OneDrive - The George Washington University\Fall 2023\CSCI 4366\Final Project\nn_f23_shellfish\../shellfish_dataset/train\b8d029497cf7bac6_jpg.rf.0180374b736435cd50e82b8ca054220c.jpg
C:\Users\black\OneDrive - The George Washington University\Fall 2023\CSCI 4366\Final Project\nn_f23_shellfish\../shellfish_dataset/train\b8d029497cf7bac6_jpg.rf.0180374b736435cd50e82b8ca054220c.jpg
C:\Users\black\OneDrive - The George Washington University\Fall 2023\CSCI 4366\Final Project\nn_f23_shellfish\../shellfish_dataset/train\faa591da3ab1fa90_jpg.rf.0b5cc08196694e1f25b4b0c8d6fdf1d8.jpg
C:\Users\black\OneDrive - The George Washington University\Fall 2023\CSCI 4366\Final Project\nn_f23_shellfish\../shellfish_dataset/train\faa591da3ab1fa90_jpg.rf.0b5cc08196694e1f25b4b0c8d6fdf1d8.jpg
C:\Users\b

MemoryError: Unable to allocate 1.98 MiB for an array with shape (416, 416, 3) and data type float32

In [ ]:
# #create the common input layer
# input_shape = (height, width, 3)
# input_layer = tf.keras.layers.Input(input_shape)

# #create the base layers
# base_layers = layers.experimental.preprocessing.Rescaling(1./255, name='bl_1')(input_layer) #normalize pixel range
# base_layers = layers.Conv2D(16, 3, padding='same', activation='relu', name='bl_2')(base_layers)
# base_layers = layers.MaxPooling2D(name='bl_3')(base_layers)
# base_layers = layers.Conv2D(32, 3, padding='same', activation='relu', name='bl_4')(base_layers)
# base_layers = layers.MaxPooling2D(name='bl_5')(base_layers)
# base_layers = layers.Conv2D(64, 3, padding='same', activation='relu', name='bl_6')(base_layers)
# base_layers = layers.MaxPooling2D(name='bl_7')(base_layers)
# base_layers = layers.Flatten(name='bl_8')(base_layers)

In [ ]:
# #create the classifier branch
# classifier_branch = layers.Dense(128, activation='relu', name='cl_1')(base_layers)
# classifier_branch = layers.Dense(num_classes, name='cl_head')(classifier_branch)

In [ ]:
# #create the localiser branch
# locator_branch = layers.Dense(128, activation='relu', name='bb_1')(base_layers)
# locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
# locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
# locator_branch = layers.Dense(4, activation='sigmoid', name='bb_head')(locator_branch)


In [ ]:
# Try building model a different way
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout

model = Sequential()

# Convolutional Phase
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape)) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(48, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# This compresses all the values to a single dimension
model.add(Flatten())

# Predictive phase using convolutional features
model.add(Dense(128, activation='relu'))

# Classification output
classification_output = Dense(num_classes, activation='softmax', name='classification_output')(model.layers[-1].output)

# Bounding box prediction output
bounding_box_output = Dense(4, activation='linear', name='bounding_box_output')(model.layers[-1].output)

model = Model(inputs=model.input, outputs=[classification_output, bounding_box_output])

losses = {
    "classification_output": "categorical_crossentropy",
    "bounding_box_output": "mse"
}

model.compile(loss=losses, optimizer='adam', metrics=['accuracy'])

In [ ]:
# model = tf.keras.Model(input_layer, outputs=[classifier_branch, locator_branch])

In [ ]:
model.summary()

In [ ]:
# for layer in model.layers:
#     if layer.name.startswith('bl_'):
#         print(layer.name)
#         layer.trainable = False

# for layer in model.layers:
#     if layer.name.startswith('bb_'):
#         print(layer.name)
#         layer.trainable = False

In [ ]:
# model.summary()

In [ ]:
# losses = {"cl_head":tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), "bb_head":tf.keras.losses.MSE}

In [ ]:
# add optimizer (Adam)
# model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])

In [ ]:
# convert data to np.array
training_epochs = 5

train_images = np.array(train_images)
train_targets = np.array(train_targets)
train_labels = np.array(train_labels)

test_images = np.array(test_images)
test_targets = np.array(test_targets)
test_labels = np.array(test_labels)

validation_images = np.array(validation_images)
validation_targets = np.array(validation_targets)
validation_labels = np.array(validation_labels)

In [ ]:
# losses = {
#     "cl_head": "categorical_crossentropy",
#     "bb_head": "mse"
# }

# model.compile(loss=losses, optimizer='adam', metrics=['accuracy'])

In [ ]:
trainTargets = {
    "classification_output": train_labels,
    "bounding_box_output": train_targets
}

testTargets = {
    "classification_output": test_labels,
    "bounding_box_output": test_targets
}

In [ ]:
# history = model.fit(train_images, trainTargets,
#                    validation_data=(test_images, testTargets),
#                    batch_size=4,
#                    epochs=training_epochs,
#                    shuffle=True,
#                    verbose=1)

history = model.fit(
    train_images,
    {"classification_output": train_labels, "bounding_box_output": train_targets},
    validation_data=(test_images, {"classification_output": test_labels, "bounding_box_output": test_targets}),
    epochs=10,
    batch_size=32
)

In [ ]:
cl_accuracy = history.history['cl_head_accuracy']
cl_val_acc = history.history['val_cl_head_accuracy']

bb_accuracy = history.history['bb_head_accuracy']
bb_val_acc = history.history['val_bb_head_accuracy']

cl_loss = history.history['cl_head_loss']
cl_val_loss = history.history['val_cl_head_loss']

bb_loss = history.history['bb_head_loss']
bb_val_loss = history.history['val_bb_head_loss']



epochs_range = range(training_epochs)


plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, cl_accuracy, label='Training Accuracy')
plt.plot(epochs_range, cl_val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Classification branch - Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, cl_loss, label='Training Loss')
plt.plot(epochs_range, cl_val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Classification branch - Loss')
plt.show()

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, bb_accuracy, label='Training Accuracy')
plt.plot(epochs_range, bb_val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Localisation branch - Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, bb_loss, label='Training Loss')
plt.plot(epochs_range, bb_val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Localisation branch - Loss')
plt.show()

print(history.history)

## Step 3: Test Model

In [ ]:
test_dir = pathlib.Path('../shellfish_dataset/test')
# test_dir = '/content/shellfish_dataset/test'
img_paths = list(test_dir.glob('*.jpg'))

plt.figure(figsize=(20, 20))
#print(len(img_paths))

for i, img_path in enumerate(img_paths):
    shellfish_img = keras.preprocessing.image.load_img(img_path, target_size=(height, width))
    img_arr = keras.preprocessing.image.img_to_array(shellfish_img)
    img_arr = tf.expand_dims(img_arr, 0)

    predictions = model.predict(img_arr)
    print(predictions)

    score = tf.nn.softmax(predictions[0][0])

    percent = int(round(100*np.max(score), 0))
    print(classes)
    print(np.argmax(score))
    print(classes[np.argmax(score)])
    class_name = classes[np.argmax(score)]
    title = "{} - {} %".format(class_name, percent)

    bbox = predictions[1][0]
    print("preds = ", type(predictions), predictions)
    bbox = [min(bbox[0], bbox[2]) * width, min(bbox[1], bbox[3]) * height, max(bbox[0], bbox[2]) * width, max(bbox[1], bbox[3]) * height]
    print("box = ", type(bbox), bbox)
    # bbox = [bbox[0] * width, bbox[1] * height, bbox[2] * width, bbox[3] * height]

    if class_name == 'Shrimp':
        shellfish_img_arr = np.array(shellfish_img)
        cv2.rectangle(shellfish_img_arr, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 255, 255), 3)
        shellfish_img_arr_rgb = cv2.cvtColor(shellfish_img_arr, cv2.COLOR_BGR2RGB)
        plt.figure()
        plt.imshow(shellfish_img_arr_rgb)
        # draw1 = Draw(shellfish_img)
        # draw1.rectangle(bbox, outline='yellow', width=2)

    if class_name == 'Lobster':
        draw1 = Draw(shellfish_img)
        draw1.rectangle(bbox, outline='red', width=2)

    if class_name == 'Crab':
        draw1 = Draw(shellfish_img)
        draw1.rectangle(bbox, outline='blue', width=2)

    ax = plt.subplot(29, 2, i+1)
    plt.imshow(shellfish_img)
    plt.title(title)
    plt.axis("off")